In [15]:
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans

import sys
sys.path.append('..')

import modules.plotting as pl
import modules.general as gen
import modules.gait_metrics as gm
import modules.assign_sides as asi
import modules.linear_algebra as lin

In [3]:
%matplotlib auto

Using matplotlib backend: MacOSX


In [4]:
file_name = '2014-12-08_P004_Post_004'

In [5]:
best_pos_dir = os.path.join('..', 'data', 'kinect', 'best pos')
hypo_dir = os.path.join('..', 'data', 'kinect', 'processed', 'hypothesis')

hypo_paths = glob.glob(os.path.join(hypo_dir, '*.pkl'))
best_pos_paths = glob.glob(os.path.join(best_pos_dir, '*.pkl'))

hypo_paths = [x for x in hypo_paths if file_name in x]
best_pos_paths = [x for x in best_pos_paths if file_name in x]

df_hypo = pd.read_pickle(hypo_paths[0])
df_head_feet = pd.read_pickle(best_pos_paths[0])

In [6]:
df_hypo.L_FOOT[200]


array([[-67.3317, -56.3553, 244.679],
       [-45.9189, -65.4187, 263.42900000000003],
       [-49.981, -65.5531, 262.126],
       [-66.737, -58.9848, 246.347],
       [-47.8853, -65.3711, 262.991],
       [-64.3456, -62.31100000000001, 249.727],
       [-65.7141, -60.9874, 248.03900000000002]], dtype=object)

## Split trial into separate passes

In [7]:
# Convert all position vectors to float type
# so they can be easily input to linear algebra functions
df_head_feet = df_head_feet.applymap(pd.to_numeric)

# Cluster frames with k means to locate the 4 walking passes
frames = df_head_feet.index.values.reshape(-1, 1)
k_means = KMeans(n_clusters=4, random_state=0).fit(frames)

# Sort labels so that the frames are in temporal order
labels = gen.map_sort(k_means.labels_)

# DataFrames for each walking pass in a trial
pass_dfs = gen.group_by_label(df_head_feet, labels)

In [8]:
df_pass = list(pass_dfs)[0]

In [9]:
_, direction_pass = gm.direction_of_pass(df_pass)

# Assign correct sides to feet
df_assigned = asi.assign_sides_pass(df_pass, direction_pass)

## Foot signal

In [10]:
signal_l = gm.foot_signal(df_assigned.L_FOOT, df_assigned.R_FOOT, direction_pass)

In [11]:
plt.figure()
plt.plot(signal_l)
plt.grid()

In [12]:
foot_dist = (df_assigned.L_FOOT - df_assigned.R_FOOT).apply(np.linalg.norm)

In [13]:
plt.figure()
plt.plot(foot_dist)
plt.grid()

## Assign DataFrame for plotting

In [78]:
df = df_assigned
dim_order = [0, 2, 1]

frames = df.index.values
frames

array([175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187,
       188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200,
       201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213,
       214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226,
       227, 228, 229, 230, 231, 232, 234, 235, 236, 237, 238, 239, 240,
       241, 246, 247, 248, 250, 251, 253, 254, 255, 256, 257, 258, 259,
       260, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 273, 274,
       277, 278, 279, 280, 282])

## View one frame

In [84]:
frame = 179

if frame in frames:
      
    fig, ax = plt.subplots()
    ax = Axes3D(fig)
    
    positions = df.loc[frame]

    points = np.stack(positions)
    points_to_plot = points[:, dim_order]

    head, foot_l, foot_r = points_to_plot

    pl.scatter3(ax, head, color='k')
    pl.scatter3(ax, foot_l, color='b')
    pl.scatter3(ax, foot_r, color='r')    

    # Plot hypotheses
    df_hypo_frame = df_hypo.loc[frame]
    pl.scatter3(ax, df_hypo_frame.L_FOOT[:, dim_order], color='w', edgecolor='k', depthshade=False)
    pl.scatter3(ax, df_hypo_frame.R_FOOT[:, dim_order], color='w', edgecolor='k', depthshade=False)

    ax.set_xlim3d(-200, 200)
    ax.set_ylim3d(100, 300)
    ax.set_zlim3d(-100, 100)

    plt.show()

## View multiple frames

In [105]:
fig, ax = plt.subplots()

ax = Axes3D(fig)

frames = df.index.values

for frame in frames:
    
    plt.cla()
    
    try:
        positions = df.loc[frame]
    except KeyError:
        continue
    
    points = np.stack(positions)
    points_to_plot = points[:, dim_order]

    head, foot_l, foot_r = points_to_plot

    pl.scatter3(ax, head, color='k')
    pl.scatter3(ax, foot_l, color='b')
    pl.scatter3(ax, foot_r, color='r')   
    
    # Plot hypotheses
    df_hypo_frame = df_hypo.loc[frame]
    pl.scatter3(ax, df_hypo_frame.L_FOOT[:, dim_order], color='w', edgecolor='k', depthshade=False)
    pl.scatter3(ax, df_hypo_frame.R_FOOT[:, dim_order], color='w', edgecolor='k', depthshade=False)

    
    ax.set_xlim3d(-200, 200)
    ax.set_ylim3d(100, 300)
    ax.set_zlim3d(-100, 100)
    
    plt.title('Frame {}'.format(frame))
    
    plt.pause(0.5)

## View 2D frames

In [105]:
%matplotlib inline

In [122]:
frame = 187

plt.figure()

if frame in frames: 
    
    positions = df.loc[frame]

    points = np.stack(positions)
    points_to_plot = points

    head, foot_l, foot_r = points_to_plot

    pl.scatter2(head, color='k')
    pl.scatter2(foot_l, color='b')
    pl.scatter2(foot_r, color='r') 
    
    plt.axis([-200, 200, -100, 80])

    plt.xlabel('X')
    plt.ylabel('Y')
    plt.grid()
    
    plt.title('Frame {}'.format(frame))

In [115]:
_, ax = plt.subplots()
frames = df.index.values

for frame in frames:
    
    plt.cla()
    
    try:
        positions = df.loc[frame]
    except KeyError:
        continue

    positions = df.loc[frame]

    points = np.stack(positions)
    points_to_plot = points

    head, foot_l, foot_r = points_to_plot

    pl.scatter2(head, color='k')
    pl.scatter2(foot_l, color='b')
    pl.scatter2(foot_r, color='r')    

    # Plot hypotheses
    df_hypo_frame = df_hypo.loc[frame]
    pl.scatter2(df_hypo_frame.L_FOOT, facecolor='none', edgecolor='k')
    pl.scatter2(df_hypo_frame.R_FOOT, facecolor='none', edgecolor='k')

    plt.axis([-200, 200, -100, 80])

    plt.xlabel('X')
    plt.ylabel('Y')
    plt.grid()

    plt.pause(0.01)